In [1]:
import os
import zipfile
import mp
from datetime import datetime
from glob import glob
from tqdm import tqdm
from huggingface_hub import HfApi
from huggingface_hub import HfFileSystem
import time

partition_size = 3e+9

In [2]:
!du -hs dedup-others

15G	dedup-others


In [3]:
files = glob('dedup-others/*.mp3')
len(files)

411637

In [4]:
def loop(files):
    files, index = files
    current_index = 0
    api = HfApi()
    fs = HfFileSystem()
    total = 0
    temp = []
    for i in tqdm(range(len(files))):
        s = os.stat(files[i]).st_size
        if s + total >= partition_size:
            part_name = f"dedup-others-{index}-{current_index}.zip"
                
            with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for f in temp:
                    zipf.write(f, arcname=f)

            while True:
                try:
                    api.upload_file(
                        path_or_fileobj=part_name,
                        path_in_repo=part_name,
                        repo_id="malaysia-ai/dedup-Malaysian-Emilia",
                        repo_type="dataset",
                    )
                    break
                except:
                    time.sleep(60)

            os.remove(part_name)
            
            current_index += 1
            temp = [files[i]]
            total = s
        else:
            temp.append(files[i])
            total += s
        
    if len(temp):
        part_name = f"dedup-others-{index}-{current_index}.zip"

        with zipfile.ZipFile(part_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in temp:
                zipf.write(f, arcname=f)

        while True:
            try:
                api.upload_file(
                    path_or_fileobj=part_name,
                    path_in_repo=part_name,
                    repo_id="malaysia-ai/dedup-Malaysian-Emilia",
                    repo_type="dataset",
                )
                break
            except:
                time.sleep(60)

        os.remove(part_name)

In [5]:
loop((files[:1000], 0))

100%|█████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 358150.80it/s]


dedup-others-0-0.zip:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

In [6]:
mp.multiprocessing(files, loop, cores = 10, returned = False)

100%|███████████████████████████████████████████████████████████████████████████████| 41163/41163 [00:00<00:00, 133537.11it/s]


dedup-others-0-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

dedup-others-3-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

dedup-others-7-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

dedup-others-6-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

...

dedup-others-2-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

dedup-others-1-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

dedup-others-5-0.zip:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

100%|████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 59554.01it/s]


dedup-others-10-0.zip:   0%|          | 0.00/192k [00:00<?, ?B/s]